# 다중 도구(Multi-tool) LangChain 에이전트 구현

## 목표
- 웹 검색, 수학 계산, 언어 요약/해석이 모두 가능한 범용 multi-tool agent 구현
- 총 3개의 독립적인 도구를 통합하여 복합적인 문제 해결
- 서울의 인구 밀도 계산 및 의미 해석 시나리오 수행

## 사용 도구
1. **SerpAPI**: 실시간 웹 검색 기능
2. **llm-math**: 자연어 수학 문제 해석 및 계산
3. **Language_Interpreter**: ChatOpenAI 기반 언어 해석, 요약 및 의미 분석

In [ ]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 필요한 패키지들 설치
packages = [
    "langchain",
    "langchain-openai", 
    "langchain-community",
    "google-search-results",
    "openai",
    "numexpr"
]

for package in packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"{package} 이미 설치됨")
    except ImportError:
        print(f"{package} 설치 중...")
        install_package(package)

langchain 이미 설치됨
langchain-openai 이미 설치됨
langchain-community 이미 설치됨
google-search-results 설치 중...
openai 이미 설치됨
numexpr 이미 설치됨


In [ ]:
# 필요한 라이브러리 import
import os
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import LLMMathChain

OPENAI_API_KEY = "my-key"
SERPAPI_API_KEY = "my-key"


# 환경변수 설정
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

## 1. 다중 도구 설정 (총 3개)

각각의 도구를 초기화하고 설정합니다:
- **Search**: SerpAPI를 활용한 웹 검색
- **Calculator**: LLM Math Chain을 활용한 수학 계산
- **Language_Interpreter**: ChatOpenAI를 활용한 언어 해석 및 의미 분석

In [3]:
# 1. LLM 초기화 (ChatOpenAI)
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo"
)

# 2. 웹 검색 도구 설정 (SerpAPI)
search = SerpAPIWrapper()

# 3. 수학 계산 도구 설정 (llm-math)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

# 4. 언어 해석/요약 도구 함수 정의 (ChatOpenAI)
def language_interpreter(query):
    """언어 해석, 요약, 의미 분석을 수행하는 함수"""
    interpretation_prompt = f"""
    다음 내용을 분석하고 해석해주세요:
    {query}
    
    명확하고 이해하기 쉽게 설명해주세요.
    """
    response = llm.invoke(interpretation_prompt)
    return response.content

# 도구들을 Tool 객체로 래핑 (총 3개)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="웹에서 최신 정보를 검색할 때 유용합니다. 실시간 데이터나 최신 통계가 필요할 때 사용하세요."
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="수학적 계산이나 복잡한 산술 연산을 수행할 때 사용합니다. 자연어로 된 수학 문제도 해결할 수 있습니다."
    ),
    Tool(
        name="Language_Interpreter",
        func=language_interpreter,
        description="텍스트의 의미를 해석하고, 요약하며, 복잡한 개념을 쉽게 설명할 때 사용합니다. 데이터나 결과의 의미를 분석할 때 유용합니다."
    )
]

print(f"총 {len(tools)}개의 도구가 준비되었습니다:")
for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}: {tool.description}")

총 3개의 도구가 준비되었습니다:
1. Search: 웹에서 최신 정보를 검색할 때 유용합니다. 실시간 데이터나 최신 통계가 필요할 때 사용하세요.
2. Calculator: 수학적 계산이나 복잡한 산술 연산을 수행할 때 사용합니다. 자연어로 된 수학 문제도 해결할 수 있습니다.
3. Language_Interpreter: 텍스트의 의미를 해석하고, 요약하며, 복잡한 개념을 쉽게 설명할 때 사용합니다. 데이터나 결과의 의미를 분석할 때 유용합니다.


## 2. Zero-Shot React Description 에이전트 초기화

요구사항에 따라 `zero-shot-react-description` 타입의 에이전트를 생성합니다.

In [4]:
# Zero-Shot React Description 에이전트 초기화
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,  # Thought, Action, Observation 단계 출력
    handle_parsing_errors=True  # 파싱 에러 처리
)

print("Multi-tool 에이전트가 성공적으로 초기화되었습니다!")
print("\n에이전트 정보:")
print(f"- 에이전트 타입: {agent.agent.llm_chain.prompt.template}")
print(f"- 사용 가능한 도구 수: {len(tools)}")
print(f"- Verbose 모드: 활성화")

Multi-tool 에이전트가 성공적으로 초기화되었습니다!

에이전트 정보:
- 에이전트 타입: Answer the following questions as best you can. You have access to the following tools:

Search(query: str, **kwargs: Any) -> str - 웹에서 최신 정보를 검색할 때 유용합니다. 실시간 데이터나 최신 통계가 필요할 때 사용하세요.
Calculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, **kwargs: Any) -> Any - 수학적 계산이나 복잡한 산술 연산을 수행할 때 사용합니다. 자연어로 된 수학 문제도 해결할 수 있습니다.
Language_Interpreter(query) - 텍스트의 의미를 해석하고, 요약하며, 복잡한 개념을 쉽게 설명할 때 사용합니다. 데이터나 결과의 의미를 분석할 때 유용합니다.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator, Language_Interpreter]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observati

/var/folders/z9/hg3hgmdn7fq2v90_5bg9xx0c0000gn/T/ipykernel_2910/1165316386.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


## 3. 성능 검증 과제 수행

**시나리오**: "서울의 최신 인구와 면적을 검색하여 인구 밀도를 계산하고, 이 수치가 갖는 의미를 간단히 설명하시오."

이 과제는 다음 3개 도구를 모두 활용하여 수행됩니다:
1. **Search (SerpAPI)**: 서울의 인구와 면적 정보 검색
2. **Calculator (llm-math)**: 인구 밀도 계산 (인구 ÷ 면적)  
3. **Language_Interpreter (ChatOpenAI)**: 계산된 수치의 의미 해석 및 설명

In [5]:
# 성능 검증 과제 실행
question = """
서울의 최신 인구와 면적을 검색하여 인구 밀도를 계산하고, 
이 수치가 갖는 의미를 간단히 설명하시오.

다음 단계로 수행해주세요:
1. 먼저 서울의 최신 인구 수를 검색하세요
2. 서울의 면적(km²)을 검색하세요  
3. 인구 밀도를 계산하세요 (인구 ÷ 면적)
4. 계산된 인구 밀도가 갖는 의미를 설명하세요
"""

print("=" * 60)
print("다중 도구 에이전트 성능 검증 시작")
print("=" * 60)

try:
    # 에이전트 실행
    final_result = agent.run(question)
    
    print("\n" + "=" * 60)
    print("최종 결과")
    print("=" * 60)
    print(final_result)
    
except Exception as e:
    print(f"에러 발생: {e}")
    print("API 키가 올바르게 설정되었는지 확인해주세요.")

다중 도구 에이전트 성능 검증 시작


> Entering new AgentExecutor chain...


/var/folders/z9/hg3hgmdn7fq2v90_5bg9xx0c0000gn/T/ipykernel_2910/3539495705.py:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  final_result = agent.run(question)


I need to search for the latest population and area of Seoul, then calculate the population density and explain its significance.

Action: Search
Action Input: "Seoul latest population"
Observation: The metro area population of Seoul in 2022 was 9,976,000, a 0.08% increase from 2021.
Thought:I have found the latest population of Seoul.

Action: Search
Action Input: "Seoul area in km²"
Observation: 605.25 km2
Thought:I have found the area of Seoul in km².

Action: Calculator
Action Input: 9976000 / 605.25

> Entering new LLMMathChain chain...
9976000 / 605.25```text
9976000 / 605.25
```
...numexpr.evaluate("9976000 / 605.25")...

Answer: 16482.44527054936
> Finished chain.

Observation: Answer: 16482.44527054936
Thought:I have calculated the population density of Seoul.

Action: Language_Interpreter
Action Input: "The population density of Seoul is 16482.45 people per square kilometer."
Observation: 서울의 인구 밀도는 1제곱킬로미터 당 16482.45명입니다. 이는 서울에 있는 각 제곱킬로미터당 약 16482.45명의 사람이 거주하고 있다는 것을 의미합니

## 4. 개별 도구 테스트

에이전트의 각 도구가 정상적으로 작동하는지 개별적으로 테스트해봅시다.

In [6]:
# 4-1. 웹 검색 도구 테스트 (Search)
print("1. 웹 검색 도구 테스트 (Search)")
print("-" * 40)

search_test = "서울 인구 2024"
search_result = agent.run(f"Search 도구를 사용해서 '{search_test}'에 대해 검색해주세요.")
print(f"검색 결과: {search_result}")

print("\n" + "=" * 60)

# 4-2. 수학 계산 도구 테스트 (Calculator)
print("2. 수학 계산 도구 테스트 (Calculator)")
print("-" * 40)

math_test = "9,720,846을 605.21로 나눈 값을 계산해주세요."
calc_result = agent.run(f"Calculator 도구를 사용해서 {math_test}")
print(f"계산 결과: {calc_result}")

print("\n" + "=" * 60)

# 4-3. 언어 해석 도구 테스트 (Language_Interpreter)
print("3. 언어 해석 도구 테스트 (Language_Interpreter)")
print("-" * 40)

interpretation_test = "인구 밀도가 16,000명/km²라는 수치의 의미"
interpret_result = agent.run(f"Language_Interpreter 도구를 사용해서 '{interpretation_test}'를 해석하고 설명해주세요.")
print(f"해석 결과: {interpret_result}")

1. 웹 검색 도구 테스트 (Search)
----------------------------------------


> Entering new AgentExecutor chain...
서울의 인구 통계에 대한 최신 정보를 얻기 위해 Search 도구를 사용해야 합니다.
Action: Search
Action Input: '서울 인구 2024'
Observation: ['2025년 9월 기준 서울특별시 인구수는 9,318,500명입니다. 전월 대비 3,363명 감소하였습니다. 데이터는 매달 최신으로 업데이트됩니다. 이 페이지를 북마크 또는 홈 화면 ...', '서울의 인구가 이동한 경기도, 인천이 서울보다는 면적이 훨씬 넓은 데다가 ... 2024년 9,331,828명. 2025년 5월 9,328,042명. 인구는 현재 행정구역이 아닌 해당 ...', '° 지역별 인구 통계자료의 단위는 천 명이며, 2024년 서울시의 인구는 939만 명임. - 인구밀도 통계자료의 단위는 명/㎢으로 2024년 서울시에는 1㎢당 15,521명이 살고 있음 ...', '2024년, 2023년, 2022년, 2021년, 2020년, 2019년, 2018년, 2017년, 2016년, 2015년 ... 서울특별시, 9,318,500, 9,318,500, 465,406, 720,713, 1,274,270, 1,457,392 ...', '2024년 서울 인구의 0.5%(4.5만 명) 유출, 전년 대비 43.0% 증가 · 경기·인천에서 서울로 전입한 인구는 26.0만 명으로 전입인구 중 60.7%를 차지 · 수도권 외 지역에서 서울 ...', '통계로 본 서울" - 인구. 인구(2024.3/4) 9,605,419명, 고령자수(2024.3/4) \u200b1,803,910명. 청소년인구(9~24세 기준)(2024.3/4) 1,331,694명, 노년부양비(2024.3/4)', '2024년 서울 인구의 0.5%(4.5만 명) 유출, 전년 대비 43.0% 증가 · 경기·인천에서 서울로 전입한 인구